# ILD CNN prediction part
    the model is rebuilt
    the weights are loaded from ILD_CNN_model_weights

In [2]:
import argparse
import numpy as np
import sklearn.metrics as metrics
import cPickle as pickle
import json
from keras.models import model_from_json
import sys
import cv2

from keras.models import Sequential
from keras.utils import np_utils 
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D,AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU

Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
import ild_helpers as H
import cnn_model4 as CNN4

In [4]:
args         = H.parse_args()                          
train_params = {
     'do' : float(args.do) if args.do else 0.5,        
     'a'  : float(args.a) if args.a else 0.3,          # Conv Layers LeakyReLU alpha param [if alpha set to 0 LeakyReLU is equivalent with ReLU]
     'k'  : int(args.k) if args.k else 4,              # Feature maps k multiplier
     's'  : float(args.s) if args.s else 1,            # Input Image rescale factor
     'pf' : float(args.pf) if args.pf else 1,          # Percentage of the pooling layer: [0,1]
     'pt' : args.pt if args.pt else 'Avg',             # Pooling type: Avg, Max
     'fp' : args.fp if args.fp else 'proportional',    # Feature maps policy: proportional, static
     'cl' : int(args.cl) if args.cl else 5,            # Number of Convolutional Layers
     'opt': args.opt if args.opt else 'Adam',          # Optimizer: SGD, Adagrad, Adam
     'obj': args.obj if args.obj else 'ce',            # Minimization Objective: mse, ce
     'patience' : args.pat if args.pat else 5,         # Patience parameter for early stoping
     'tolerance': args.tol if args.tol else 1.005,     # Tolerance parameter for early stoping [default: 1.005, checks if > 0.5%]
     'res_alias': args.csv if args.csv else 'res'      # csv results filename alias
}

In [5]:
model = H.load_model()

In [6]:
model.compile(optimizer='Adam', loss=CNN4.get_Obj(train_params['obj']))

In [18]:
(X_test, y_test) = H.load_testdata()
X_test = X_test[0:10]
y_test = y_test[0:10]

In [19]:
y_test.shape
y_test

array([[ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.]])

In [20]:
# predict and check classification and probabilities are the same

classes = model.predict_classes(X_test, batch_size=10)
proba = model.predict_proba(X_test, batch_size=10)

10/10 [==============================] - 0s


In [21]:
classes

array([1, 4, 4, 3, 3, 4, 3, 2, 1, 5])

In [22]:
proba

array([[  2.33090643e-04,   9.98344958e-01,   1.41310238e-03,
          1.69823017e-10,   1.29506939e-09,   8.81856249e-06],
       [  9.74275613e-16,   3.19316023e-05,   4.29552429e-08,
          1.22379254e-06,   9.99966621e-01,   1.81914231e-07],
       [  0.00000000e+00,   8.29829797e-23,   5.99833414e-21,
          2.90690106e-18,   1.00000000e+00,   4.47039583e-33],
       [  2.56549320e-05,   4.71907295e-02,   1.47910416e-01,
          4.33063507e-01,   3.71788919e-01,   2.07721332e-05],
       [  2.56549320e-05,   4.71907295e-02,   1.47910416e-01,
          4.33063507e-01,   3.71788919e-01,   2.07721332e-05],
       [  1.76317834e-33,   5.05142315e-13,   3.06530301e-10,
          5.45444467e-10,   1.00000000e+00,   1.13258984e-17],
       [  7.18908548e-13,   3.47596620e-06,   3.76264211e-07,
          9.99628305e-01,   3.67829663e-04,   5.53017054e-15],
       [  4.63859105e-05,   2.99533546e-01,   5.97326577e-01,
          3.46729124e-04,   8.71171802e-02,   1.56296194e-02],
